In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity
import statsmodels.api as sm
import numpy as np

# Cargar los datos
df = pd.read_csv('Employee Attrition.csv')

# Seleccionar columnas numéricas
numeric_columns = [
    'satisfaction_level', 'last_evaluation', 'number_project', 
    'average_montly_hours', 'time_spend_company', 'Work_accident', 
    'promotion_last_5years'
]

# Imputar valores faltantes con la media
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())

# Estandarizar los datos
scaler = StandardScaler()
df_numeric = scaler.fit_transform(df[numeric_columns])

# Calcular KMO y prueba de esfericidad de Bartlett
kmo_all, kmo_model = calculate_kmo(df_numeric)
bartlett_chi_square, bartlett_p_value = calculate_bartlett_sphericity(df_numeric)

print(f"KMO Model: {kmo_model}")
print(f"Bartlett's Test: Chi-Square = {bartlett_chi_square}, p-value = {bartlett_p_value}")

# Realizar Análisis Factorial Exploratorio (EFA) con diferentes números de factores
for n_factors in range(1, 4):
    fa = FactorAnalyzer(n_factors=n_factors, rotation='varimax')
    fa.fit(df_numeric)
    
    # Verificar la varianza explicada por cada factor
    variance = fa.get_factor_variance()
    print(f"Number of Factors: {n_factors}")
    print(f"Factor Variance: {variance}")
    
    # Obtener las cargas factoriales
    loadings = fa.loadings_
    print(f"Factor Loadings for {n_factors} factors:\n{loadings}\n")

# Realizar Análisis Factorial Confirmatorio (CFA) utilizando statsmodels
X = df_numeric[:, 1:]
y = df_numeric[:, 0]
model = sm.OLS(y, X)
results = model.fit()

print(f"CFA Results: {results.summary()}")



KMO Model: 0.6270063648376561
Bartlett's Test: Chi-Square = 7709.041435943096, p-value = 0.0
Number of Factors: 1
Factor Variance: (array([1.19500725]), array([0.17071532]), array([0.17071532]))
Factor Loadings for 1 factors:
[[-0.07185504]
 [ 0.51227766]
 [ 0.70261333]
 [ 0.60603996]
 [ 0.25740947]
 [-0.01393851]
 [ 0.00344254]]

Number of Factors: 2
Factor Variance: (array([1.20419167, 1.03032241]), array([0.17202738, 0.14718892]), array([0.17202738, 0.3192163 ]))
Factor Loadings for 2 factors:
[[-0.07725378  0.99453615]
 [ 0.54052461  0.14729983]
 [ 0.68843291 -0.09248856]
 [ 0.60654108  0.02935501]
 [ 0.25314242 -0.08088028]
 [-0.01124854  0.05589024]
 [ 0.00413682  0.02101988]]

Number of Factors: 3
Factor Variance: (array([1.20036474, 0.83684777, 0.18931058]), array([0.17148068, 0.11954968, 0.02704437]), array([0.17148068, 0.29103036, 0.31807473]))
Factor Loadings for 3 factors:
[[-0.01484192  0.88102804  0.12628747]
 [ 0.55376656  0.12932844 -0.01220089]
 [ 0.67896673 -0.1505324

d:\coco\Lib\site-packages\factor_analyzer\factor_analyzer.py:663: UserWarning: No rotation will be performed when the number of factors equals 1.
  warnings.warn(


In [2]:
import pandas as pd
from factor_analyzer import FactorAnalyzer, calculate_kmo
from scipy.stats import bartlett
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Cargar el conjunto de datos
file_path = "Factor-Hair-Revised.csv"
df = pd.read_csv(file_path)

# Seleccionar las columnas relevantes para el análisis
data = df[['ProdQual', 'Ecom', 'TechSup', 'CompRes', 'Advertising', 
           'ProdLine', 'SalesFImage', 'ComPricing', 'WartyClaim', 'OrdBilling', 
           'DelSpeed', 'Satisfaction']]

# Imputar valores faltantes con la mediana de cada columna
data.fillna(data.median(), inplace=True)

# Normalizar los datos
data = (data - data.mean()) / data.std()

# Verificar si aún hay valores NaN o infinitos
print(data.isna().sum())
print(np.isinf(data).sum())

# Cálculo de la medida KMO y prueba de Esfericidad de Bartlett
kmo_all, kmo_model = calculate_kmo(data)
print(f'KMO: {kmo_model:.2f}')

chi2, p_value = bartlett(*data.values.T)
print(f'Chi2: {chi2:.2f}, p-value: {p_value:.2f}')

# Análisis Factorial Exploratorio (EFA)
fa = FactorAnalyzer(n_factors=3, rotation='varimax')
fa.fit(data)

# Cargas factoriales
loadings = fa.loadings_
print('Cargas factoriales:')
print(loadings)

# Simulación de datos para CFA
data['Factor1'] = np.dot(data.iloc[:, :4], loadings[:4, 0])
data['Factor2'] = np.dot(data.iloc[:, 4:8], loadings[4:8, 1])
data['Factor3'] = np.dot(data.iloc[:, 8:12], loadings[8:12, 2])

# Modelo CFA
model = ols('Factor1 ~ ProdQual + Ecom + TechSup + CompRes', data=data).fit()
print(model.summary())

# Índices de ajuste para CFA
def calc_fit_indices(model):
    chi2 = model.ssr / model.df_resid
    rmsea = np.sqrt(chi2 / (model.nobs - model.df_model - 1))
    return {
        'Chi2': chi2,
        'RMSEA': rmsea,
        'CFI': sm.regression.linear_model.OLS(data['Factor1'], sm.add_constant(data.iloc[:, :5])).fit().rsquared
    }

fit_indices = calc_fit_indices(model)
print('Índices de ajuste:')
print(fit_indices)


ProdQual        0
Ecom            0
TechSup         0
CompRes         0
Advertising     0
ProdLine        0
SalesFImage     0
ComPricing      0
WartyClaim      0
OrdBilling      0
DelSpeed        0
Satisfaction    0
dtype: int64
ProdQual        0
Ecom            0
TechSup         0
CompRes         0
Advertising     0
ProdLine        0
SalesFImage     0
ComPricing      0
WartyClaim      0
OrdBilling      0
DelSpeed        0
Satisfaction    0
dtype: int64
KMO: 0.66
Chi2: -0.00, p-value: 1.00
Cargas factoriales:
[[ 0.32306213 -0.2705285   0.17073745]
 [ 0.08793391  0.75735691  0.03733059]
 [ 0.03258596 -0.02493452  0.86665472]
 [ 0.86150135  0.14096621  0.01299146]
 [ 0.18514071  0.52780551 -0.0398777 ]
 [ 0.74946102 -0.25099871  0.24793119]
 [ 0.16193573  0.96333097  0.07887095]
 [-0.28915703  0.396071   -0.32104411]
 [ 0.11782011  0.04917139  0.85874763]
 [ 0.7326499   0.14647668  0.04439584]
 [ 0.86601902  0.20310792 -0.04748778]
 [ 0.71739946  0.26248612  0.16065133]]
                

C:\Users\MARCO\AppData\Local\Temp\ipykernel_8188\3008084614.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fillna(data.median(), inplace=True)


In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity
import numpy as np

# Cargar los datos
df = pd.read_csv('Employee Attrition.csv')

# Seleccionar columnas numéricas
numeric_columns = [
    'satisfaction_level', 'last_evaluation', 'number_project', 
    'average_montly_hours', 'time_spend_company', 'Work_accident', 
    'promotion_last_5years'
]

# Imputar valores faltantes con la media
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())

# Estandarizar los datos
scaler = StandardScaler()
df_numeric = scaler.fit_transform(df[numeric_columns])

# Calcular KMO y prueba de esfericidad de Bartlett
kmo_all, kmo_model = calculate_kmo(df_numeric)
bartlett_chi_square, bartlett_p_value = calculate_bartlett_sphericity(df_numeric)

# Realizar el análisis factorial con rotación Varimax
fa = FactorAnalyzer(n_factors=3, rotation='varimax')
fa.fit(df_numeric)

# Obtener las cargas factoriales
loadings = fa.loadings_

# Obtener los eigenvalores
eigen_values, vectors = fa.get_eigenvalues()

# Resultados
print(f"KMO Model: {kmo_model}")
print(f"Bartlett's Test: Chi-Square = {bartlett_chi_square}, p-value = {bartlett_p_value}")
print("Cargas Factoriales para 3 factores:")
print(loadings)


KMO Model: 0.6270063648376561
Bartlett's Test: Chi-Square = 7709.041435943096, p-value = 0.0
Cargas Factoriales para 3 factores:
[[-0.01484192  0.88102804  0.12628747]
 [ 0.55376656  0.12932844 -0.01220089]
 [ 0.67896673 -0.15053245 -0.01401896]
 [ 0.607044   -0.00347354 -0.04422512]
 [ 0.25281103 -0.13529178  0.17532667]
 [-0.00799381  0.04964397  0.10375166]
 [ 0.00339249 -0.02170329  0.35994021]]
